In [114]:
import pandas as pd
import numpy as np
import csv

# Import data

In [9]:
DATA_FOLDER = "competition-data/"
DATA_TEST = "test.csv"
DATA_TRAIN = "train.csv"

In [119]:
train_path = DATA_FOLDER + DATA_TRAIN
test_path = DATA_FOLDER + DATA_TEST
test_data = pd.read_csv(test_path, na_values=[-999.000])
train_data = pd.read_csv(train_path, na_values=[-999.000])

In [24]:
train_data.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,211886.000000,250000.000000,250000.000000,250000.000000,72543.000000,72543.000000,72543.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,150087.000000,150087.000000,150087.000000,72543.000000,72543.000000,72543.000000,250000.000000
mean,224999.500000,121.858528,49.239819,81.181982,57.895962,2.403735,371.783360,-0.821688,2.373100,18.917332,...,-0.010119,209.797178,0.979176,84.822105,-0.003275,-0.012393,57.679474,-0.011845,-0.001582,73.064591
std,72168.927986,57.298157,35.344886,40.828691,63.655682,1.742226,397.699325,3.584362,0.782911,22.273494,...,1.812223,126.499506,0.977426,60.662276,1.784546,1.813385,31.985782,2.031743,1.816950,98.015662
min,100000.000000,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,162499.750000,91.885250,19.241000,59.388750,14.068750,0.882500,111.977000,-2.629000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,44.422500,-1.342000,-1.584000,37.312000,-1.612000,-1.576500,0.000000
50%,224999.500000,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,287499.250000,135.482000,73.598000,92.259000,79.169000,3.690000,478.226000,0.958000,2.961000,27.591000,...,1.561000,263.379250,2.000000,103.342000,1.336000,1.562000,66.637000,1.589500,1.576000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


# Clean data

In [121]:
def sanitize_data(train_data, test_data):
    complete_data = pd.concat([test_data, train_data])
    complete_data_mean = complete_data.mean()
    sanitized_train_data = train_data.fillna(complete_data_mean)
    sanitized_test_data = test_data.fillna(complete_data_mean)
    
    return (sanitized_train_data, sanitized_test_data)

In [140]:
(sanitized_train_data, sanitized_test_data) = sanitize_data(train_data, test_data)

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470000,51.655,97.827,27.980,0.910000,124.71100,2.66600,3.064,...,-0.277,258.733,2,67.435000,2.150000,0.444000,46.062000,1.240000,-2.47500,113.497
1,100001,b,160.937000,68.768,103.235,48.146,2.404626,372.18105,-0.82874,3.473,...,-1.916,164.546,1,46.226000,0.725000,1.158000,57.810286,-0.006669,-0.01047,46.226
2,100002,b,121.867697,162.172,125.953,35.635,2.404626,372.18105,-0.82874,3.148,...,-2.186,260.414,1,44.251000,2.053000,-2.028000,57.810286,-0.006669,-0.01047,44.251
3,100003,b,143.905000,81.417,80.943,0.414,2.404626,372.18105,-0.82874,3.310,...,0.060,86.062,0,84.904285,-0.001248,-0.018856,57.810286,-0.006669,-0.01047,0.000
4,100004,b,175.864000,16.915,134.805,16.405,2.404626,372.18105,-0.82874,3.891,...,-0.871,53.131,0,84.904285,-0.001248,-0.018856,57.810286,-0.006669,-0.01047,0.000


In [ ]:
sanitized_train_data.head()

In [150]:
features = sanitized_train_data[sanitized_train_data.columns[2:-1]].apply(lambda x: x.corr(y)).sort_values().where(lambda x : abs(x) > 0.1).dropna().index

In [152]:
sanitized_train_data = sanitized_train_data[features]
sanitized_test_data = sanitized_test_data[features]

In [126]:
def extract_tx(sanitized_train_data, sanitized_test_data):
    tx_train = sanitized_train_data[sanitized_train_data.columns[2:]].values
    tx_test = sanitized_test_data[sanitized_test_data.columns[2:]].values
    
    return (tx_train, tx_test)

In [158]:
(tx_train, tx_test) = extract_tx(sanitized_train_data, sanitized_test_data)

In [129]:
def rename_y(higgs_value, background_value, named_y):
    return pd.Series(list(map(lambda x : higgs_value if x=='s' else background_value, named_y)))

In [159]:
y = rename_y(1, -1, train_data['Prediction'])


In [167]:
def init_w(tx):
    return np.random.rand(tx.shape[1])

In [168]:
initial_w = init_w(tx_train)
initial_w

array([ 0.27119023,  0.26232572,  0.51634521,  0.8980604 ,  0.44783269,
        0.38228516,  0.02285752,  0.61729405,  0.27447737,  0.02219778])

# Algorithms

In [100]:
def compute_gradient(y, tx, w):
    # Compute the gradient and loss using MSE

    N = len(y)
    e = y - tx.dot(w)
    gradient = -1/N * tx.T.dot(e)

    return gradient

def compute_loss(y, tx, w):
    # Compute the gradient and loss using MSE

    N = len(y)
    e = y - tx.dot(w)
    loss = 1/(2*N) * np.sum(e**2, axis=0)

    return loss

def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    # Linear regression using gradient descent

    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        # update w by gradient
        w = w - gamma * gradient

    return (w, loss)

def standardize(x):
    
    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    
    return std_data

def least_squares(y, tx):
    # Least squares regression using normal equations


    w = np.linalg.solve(tx.T.dot(tx), tx.T.dot(y))
    loss = compute_loss(y, tx, w)

    return (w, loss)

In [163]:
tx_train = standardize(tx_train)
tx_test = standardize(tx_test)

In [98]:
least_squares_GD(y, tx, initial_w, 500, 0.5)

(array([  7.23985034e+166,  -2.99665935e+167,  -5.08584338e+166,
          1.44830388e+168,  -7.74323581e+166,   2.50864049e+167,
         -1.03566812e+166,  -8.47630359e+167,   7.08260564e+167,
          1.53923061e+168,   1.21621404e+167,   8.05022440e+167,
         -2.06677819e+166,   7.06282445e+167,   5.67975410e+165,
          1.02765798e+166,   6.10567546e+167,   1.70429176e+166,
         -5.90288228e+165,   1.00455549e+168,   1.38395074e+166,
          1.46273071e+168,   1.16594627e+168,   1.16733312e+168,
          1.99108625e+165,   3.57014941e+165,   6.78358962e+167,
          2.08139291e+164,  -1.17693522e+166,   1.51809422e+168]), inf)

In [162]:
(w, loss) = least_squares(y, tx_train)

In [106]:
def predict_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = -1
    y_pred[np.where(y_pred > 0)] = 1
    
    return y_pred

In [171]:
y_pred = predict_labels(w, tx_test)
y_pred

(array([], dtype=int64),)

# Submission

In [111]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [166]:
create_csv_submission(sanitized_data_test['Id'], y_pred, "test2.csv")

Test n : algorithm / features / y / w
- - - - - - - - - - - - - - - - - - - 
Test 1 : least_squares / all features standardized / y = -1,1 / random init_w
Test 2 : least_squares / corr > 0.1 features standardized / y = -1,1 / random init_w